# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='fd6d3a34-35b4-472b-a9f2-3201df4ee13e', ...)


INFO:tensorflow:Restoring parameters from model_runs/bitty_blobs/classifymodel_5000.ckpt


ValueError: Cannot feed value of shape (1, 10000) for Tensor 'Placeholder:0', which has shape '(100, 10000)'